In [6]:
#New file to vectorise the reviews to perform the data science on them

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


<b><h2>Preprocessing of any inputted data

In [8]:
def convert_lower_case(data):
    return np.char.lower(data)

In [9]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [10]:

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [11]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [12]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [13]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [5]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

<b><h2>Handling of the review data and vectorising of the text

In [14]:
df = pd.read_csv("../data/review_dataset.csv")
df = pd.DataFrame(df)
df.dropna()

,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review
0,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,have to say this is the best salon ive ever be...,1.652418e+09,2022-05-13 05:07:20,say best salon ive ever 4 weeks lifting acryli...,say best salon ive ever 4 week lifting acryli...
1,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,nian is incredible ive been coming here for a ...,1.661441e+09,2022-08-25 15:22:45,nian incredible ive coming couple months every...,nian incredible ive coming couple month every...
2,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i got a new set of gels from tee who was so so...,1.658412e+09,2022-07-21 14:04:52,got new set gels tee lovely chat helped choose...,got new set gel tee lovely chat helped choose...
3,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i have been coming to this salon since i moved...,1.660734e+09,2022-08-17 10:57:00,coming salon since moved birmingham took find ...,coming salon since moved birmingham took find...
4,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i had my nails done for the first time and i c...,1.659646e+09,2022-08-04 20:41:01,nails done first time say place super cozy gir...,nail done first time say place super cozy gir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22374,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,just had ford ranger polished so pleased with ...,1.654364e+09,2022-06-04 17:31:51,ford ranger polished pleased results vehicle l...,ford ranger polished pleased result vehicle l...
22375,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,the name says it all fantastic service great p...,1.641225e+09,2022-01-03 15:44:22,name says fantastic service great prices top q...,name say fantastic service great price top qu...
22376,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,a brilliant super hand car wash drove away spa...,1.652984e+09,2022-05-19 18:06:27,brilliant super hand car wash drove away spark...,brilliant super hand car wash drove away spar...
22377,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,always happy when my black and yellow fiesta c...,1.654781e+09,2022-06-09 13:28:00,always happy black yellow fiesta comes gleamin...,always happy black yellow fiesta come gleamin...


In [15]:
'''Unique words separated into a dictionary'''
test_dataset = df["lemmatized review"]
DF = {}
def keywords(dataset):
    for i in range(len(dataset["lemmatized review"])):
        value = dataset["lemmatized review"][i].strip().split(" ")
        for w in value:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}


keywords(df)
    



In [16]:
for i in DF:
    DF[i] = len(DF[i])

print(len(DF))
DF

total_vocab_size = len(DF)

21113


In [17]:
total_vocab = [x for x in DF]

In [29]:
def review_frequency(word):
    c = 0
    try:
        c=DF[word]
    except:
        pass
    return c

In [30]:
from collections import Counter
N = len(df["lemmatized review"])
review = 0
tf_idf = {}
tfidf_dataset = df["lemmatized review"]
for i in range(N):
    tokens = tfidf_dataset[i].strip().split(" ")
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token]/words_count
        document_frequency = review_freq(token)
        idf = np.log((N+1)/(document_frequency+1))
        tf_idf[review,token] = tf*idf
    review +=1






In [248]:
#tf_idf[(,word)]

In [31]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [32]:
def matching_score(k,query):
    preprocessed_query = preprocess(query)
    query_tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(query_tokens)

    query_weights={}

    for key in tf_idf:
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    query_weights = sorted(query_weights.items(),key=lambda x: x[1], reverse=True)

    print("")

    l = []

    for i in query_weights[:10]:
        l.append(i[0])

    print(l)

matching_score(10,"right so i washed my car in here few weeks back and i admitted that the car was not in the best estate i gave them 15 tips on top of the 10 wash now i went in and they said that the was costs 15 pounds because last time the car was dirty excuse me is not that a car wash should i wash it myself before i come there and only give you 10 just for entering in your premises i have never washed my car without giving a generous tip no matter the cleanliness of the cartoday you would have received tip today as well and that would have been more than what you said its going to be extra i wish your business to be exactly like you treat your customer zero to none")


Matching Score

Query: right so i washed my car in here few weeks back and i admitted that the car was not in the best estate i gave them 15 tips on top of the 10 wash now i went in and they said that the was costs 15 pounds because last time the car was dirty excuse me is not that a car wash should i wash it myself before i come there and only give you 10 just for entering in your premises i have never washed my car without giving a generous tip no matter the cleanliness of the cartoday you would have received tip today as well and that would have been more than what you said its going to be extra i wish your business to be exactly like you treat your customer zero to none

['right', 'wash', 'car', 'week', 'back', 'admit', 'car', 'best', 'estat', 'gave', 'fifteen', 'tip', 'top', 'ten', 'wash', 'went', 'said', 'cost', 'fifteen', 'pound', 'last', 'time', 'car', 'dirti', 'excu', 'car', 'wash', 'wash', 'come', 'give', 'ten', 'enter', 'premi', 'never', 'wash', 'car', 'without', 'give', 'ge

In [33]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [34]:
def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))

    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}

    for token in np.unique(tokens):

        tf = counter[token]/words_count
        document_frequency = review_frequency(token)
        idf = math.log((N+1)/(document_frequency+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [35]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Cosine Similarity

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['without', 'drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'next', 'slat', 'oak', 'bench', 'canist', 'still', 'clutch', 'survey']


/var/folders/1p/q8_71q2n5kx_1k4d5lf854km0000gp/T/ipykernel_24095/3911138133.py:2: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))



[22378  7462  7454  7455  7456  7457  7458  7459  7460  7461]
